In [11]:
import torch
import numpy as np
import sys
import matplotlib.pyplot as plt
import torchvision
from torchvision import transforms
%matplotlib auto

Using matplotlib backend: Qt5Agg


In [23]:
w1.shape,b1.shape,w2.shape,b2.shape

(torch.Size([784, 256]),
 torch.Size([256]),
 torch.Size([256, 10]),
 torch.Size([256]))

In [14]:
def relu(x):
    return torch.max(input=x,other=torch.tensor(0.0))

In [31]:
def net(x):
    x=x.view((-1,num_inputs))         # -1,784
    h=relu(torch.matmul(x,w1)+b1)     #-1,784 * 784*256 = -1 ,256 +256
#     print('h.shape : ',h.shape)
#     print('w2.shape : ',w2.shape)
    return torch.matmul(h,w2)+b2      #-1* 256 * 256 * 10 = -1, 10 + 10


In [16]:
loss=torch.nn.CrossEntropyLoss()

In [35]:
def sgd(x,lm,params):
    for p in params:
        p.data -= lm * p.grad /len(x)

In [40]:
def evaluate_accuracy(data_iter, net):
    acc_sum, n = 0.0, 0
    for x, y in data_iter:
        acc_sum += (net(x).argmax(dim=1) == y).float().sum().item()
        n += y.shape[0]
    return acc_sum / n

In [17]:
num_epochs, lr = 5, 0.1


def train_ch3(net,
              train_iter,
              test_iter,
              loss,
              num_epochs,
              batch_size,
              params=None,
              lr=None,
              optimizer=None):
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n = 0.0, 0.0, 0
        for x, y in train_iter:
            y_hat = net(x)
            l = loss(y_hat, y).sum()
#             print('-----------')
#             print('y_hat: \n',y_hat)
#             print('y: \n',y)
#             print(l.item())
#             print('-----------')
            
            if params is not None and params[0].grad is not None:
                for param in params:
                    param.grad.data.zero_()
            l.backward()
#             if optimizer is None:
            sgd(x, lr, params)

            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(dim=1) == y).float().sum().item()
            n += y.shape[0]
        test_acc = evaluate_accuracy(test_iter, net)
        print('epoch %d ,loss %.4f,train acc %.3f,test acc %.3f' %
              (epoch + 1, train_l_sum / n, train_acc_sum / n, test_acc))

In [41]:
batch_size=256
mnist_train=torchvision.datasets.FashionMNIST(
    root='F:\study\ml\DataSet\FashionMNIST',train=True,
    download=True,transform=transforms.ToTensor())
mnist_test=torchvision.datasets.FashionMNIST(
    root='F:\study\ml\DataSet\FashionMNIST',train=False,
    download=True,transform=transforms.ToTensor())

In [42]:
train_iter=torch.utils.data.DataLoader(mnist_train,batch_size=batch_size,
                                      shuffle=True,num_workers=4)
test_iter=torch.utils.data.DataLoader(mnist_test,batch_size=batch_size,
                                      shuffle=True,num_workers=4)

In [43]:
num_inputs,num_outputs,num_hiddens=784,10,256
w1=torch.tensor(np.random.normal(0,0.01,(num_inputs,num_hiddens)),dtype=torch.float)
b1=torch.zeros(num_hiddens,dtype=torch.float)
w2=torch.tensor(np.random.normal(0,0.01,(num_hiddens,num_outputs)),dtype=torch.float)
b2=torch.zeros(num_outputs,dtype=torch.float)

params=[w1,b1,w2,b2]
for param in params:
    param.requires_grad_(True)

In [44]:
train_ch3(net, train_iter, test_iter, loss, num_epochs, batch_size,
          params, lr)

epoch 1 ,loss 0.0090,train acc 0.108,test acc 0.124
epoch 2 ,loss 0.0090,train acc 0.132,test acc 0.150
epoch 3 ,loss 0.0090,train acc 0.153,test acc 0.164
epoch 4 ,loss 0.0089,train acc 0.166,test acc 0.180
epoch 5 ,loss 0.0089,train acc 0.178,test acc 0.192
